# Opening a New Shopping Mall in Bangalore, India

1. Build a dataframe of neighborhoods in Bangalore, India  by web scraping the data from Wikipedia page
2. Get the geographical coordinates of the neighborhoods
3. Obtain the venue data for the neighborhoods from Foursquare API
4. Explore and cluster the neighborhoods
5. Select the best cluster to open a new Multiplex.

In [1]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:

# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [60]:
# create a new DataFrame from the list
bl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

bl_df.head()

,Neighborhood
0,List of areas in Bangalore Cantonment
1,List of areas in Bengaluru Pete
2,List of neighbourhoods in Bangalore
3,Adugodi
4,"Agara, Bangalore"


In [61]:
bl_df.drop([0,1,2],inplace=True)
bl_df.reset_index(drop=True,inplace=True)

In [62]:
# print the number of rows of the dataframe
bl_df.shape

(129, 1)

In [63]:
# define a function to get coordinates
def get_latlng(neighborhood):
    print(neighborhood)
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{},Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [64]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in bl_df["Neighborhood"].tolist() ]

Adugodi
Agara, Bangalore
Ananthnagar
Anjanapura
Arekere
Austin Town
Babusapalya
Bagalur, Bangalore Urban
Bahubalinagar
Banashankari
Banaswadi
Basavanagudi
Basaveshwaranagar
Bellandur
Bharathnagar
Bilekahalli
Bommanahalli
Bommasandra
Brookefield
BTM Layout
Byatarayanapura
Carmelaram
Chamrajpet
Chandapura
Chickpet
Chikkalasandra
Cooke Town
Cox Town, Bangalore
CV Raman Nagar
Dasarahalli
Devarachikkanahalli
Dollars Colony
Domlur
Ejipura
Electronic City
Fraser Town, Bangalore
Gandhi Bazaar
Gandhi Nagar, Bangalore
Ganganagar, Bangalore
Girinagar
Gowdanapalya
Hanumanthanagar, Bengaluru
HBR Layout
Hebbagodi
Hebbal
Hoodi
Horamavu
HSR Layout
Hulimavu
Immadihalli
Indiranagar
Ittamadu
J. P. Nagar
Jakkur
Jalahalli
Jayanagar, Bangalore
Jeevanbheema Nagar
Jigani
Kaggadasapura
Kalasipalyam
Kalikanagar
Kalkere
Kalyan Nagar
Kamakshipalya
Kammanahalli
Kengeri
Kodigehalli
Kodihalli, Bangalore
Konanakunte
Koppa gate
Koramangala
Krishnarajapuram
Kumaraswamy Layout
Kundalahalli
Lingarajapuram
Madiwala
Mahade

In [65]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [66]:
# merge the coordinates into the original dataframe
bl_df['Latitude'] = df_coords['Latitude']
bl_df['Longitude'] = df_coords['Longitude']

In [67]:
# get the coordinates of Bangalore
address = 'bangalore, karnataka'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9791198, 77.5912997.


In [68]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(bl_df['Latitude'], bl_df['Longitude'], bl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [69]:
# define Foursquare Credentials and Version
CLIENT_ID = 'DRQ2IAYFFN510PNIRCCQXCTIGZGUYRMLD0NSTUREW5HUN10D' # your Foursquare ID
CLIENT_SECRET = 'EMNUFLHR5HUS0MYYK5UFQ3VYP23E4NNFB32JVPLHFICLJMBU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DRQ2IAYFFN510PNIRCCQXCTIGZGUYRMLD0NSTUREW5HUN10D
CLIENT_SECRET:EMNUFLHR5HUS0MYYK5UFQ3VYP23E4NNFB32JVPLHFICLJMBU


In [70]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(bl_df['Latitude'], bl_df['Longitude'], bl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [71]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6820, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adugodi,12.94402,77.608,PVR IMAX,12.934595,77.611321,Movie Theater
1,Adugodi,12.94402,77.608,Tommy Hilfiger,12.934552,77.611347,Clothing Store
2,Adugodi,12.94402,77.608,Truffles Ice & Spice,12.933443,77.614265,Burger Joint
3,Adugodi,12.94402,77.608,PVR Cinemas,12.934389,77.611184,Multiplex
4,Adugodi,12.94402,77.608,Zingron - Naga Kitchen,12.936271,77.615051,Indian Restaurant


In [72]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,100,100,100,100,100,100
Ananthnagar,26,26,26,26,26,26
Anjanapura,4,4,4,4,4,4
Arekere,75,75,75,75,75,75
Austin Town,100,100,100,100,100,100
BTM Layout,85,85,85,85,85,85
Babusapalya,48,48,48,48,48,48
"Bagalur, Bangalore Urban",9,9,9,9,9,9
Bahubalinagar,22,22,22,22,22,22


In [73]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 233 uniques categories.


In [74]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Movie Theater', 'Clothing Store', 'Burger Joint', 'Multiplex',
       'Indian Restaurant', 'Financial or Legal Service', 'Lounge',
       'Creperie', 'Brewery', 'Donut Shop', 'Breakfast Spot', 'Café',
       'Tea Room', 'Shopping Mall', 'Bar', 'Gym', 'Ice Cream Shop',
       'Dessert Shop', 'Juice Bar', 'Coffee Shop', 'Mobile Phone Shop',
       'Italian Restaurant', 'Bookstore', 'Arcade', 'Bakery',
       'Chinese Restaurant', 'Indian Sweet Shop', 'Gaming Cafe',
       'Yoga Studio', 'Smoke Shop', "Men's Store", 'Punjabi Restaurant',
       'Andhra Restaurant', 'Paper / Office Supplies Store',
       'Pizza Place', 'Thai Restaurant', 'Kerala Restaurant',
       'Cosmetics Shop', 'Mexican Restaurant', 'Salon / Barbershop',
       'Comfort Food Restaurant', 'Food Court', 'Fast Food Restaurant',
       'Bowling Alley', 'Middle Eastern Restaurant', 'Persian Restaurant',
       'Eastern European Restaurant', 'Tibetan Restaurant',
       'Electronics Store', 'Liquor Store'], dtype=o

In [75]:
# one hot encoding
bl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bl_onehot.columns[-1]] + list(bl_onehot.columns[:-1])
bl_onehot = bl_onehot[fixed_columns]

print(bl_onehot.shape)
bl_onehot.head()

(6820, 234)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [76]:
bl_grouped = bl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(bl_grouped.shape)
bl_grouped

(128, 234)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outlet Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0.000000,0.000000,0.00000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.020000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.040000,0.000000,0.010000,0.000000,0.030000,0.01,0.000000,0.000000,0.01,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0000

In [77]:
len(bl_grouped[bl_grouped["Multiplex"] > 0])

39

In [78]:
bl_mall = bl_grouped[["Neighborhoods","Multiplex"]]

In [79]:
kclusters = 3

bl_clustering = bl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 0, 1, 0, 1, 1, 1, 1])

In [80]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
bl_merged = bl_mall.copy()

# add clustering labels
bl_merged["Cluster Labels"] = kmeans.labels_

In [81]:
bl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bl_merged.head()

,Neighborhood,Multiplex,Cluster Labels
0,Adugodi,0.020000,0
1,Ananthnagar,0.000000,1
2,Anjanapura,0.000000,1
3,Arekere,0.026667,0
4,Austin Town,0.000000,1


In [82]:

bl_merged = bl_merged.join(bl_df.set_index("Neighborhood"), on="Neighborhood")

print(bl_merged.shape)
bl_merged.head() # check the last columns!

(128, 5)


,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,Adugodi,0.020000,0,12.94402,77.60800
1,Ananthnagar,0.000000,1,12.95408,77.54135
2,Anjanapura,0.000000,1,12.85811,77.55909
3,Arekere,0.026667,0,12.88568,77.59668
4,Austin Town,0.000000,1,12.96348,77.61297


In [83]:

# sort the results by Cluster Labels
print(bl_merged.shape)
bl_merged.sort_values(["Cluster Labels"], inplace=True)
bl_merged

(128, 5)


,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,Adugodi,0.020000,0,12.944020,77.608000
90,Peenya,0.041667,0,13.031850,77.526790
87,Nayandahalli,0.050000,0,12.942050,77.521000
86,Nandini Layout,0.034483,0,13.014810,77.538910
84,Muthyalanagar,0.025641,0,13.041030,77.551480
47,Hulimavu,0.034483,0,12.880640,77.601470
106,"Siddapura, Bangalore",0.035714,0,12.956130,77.731960
48,Immadihalli,0.062500,0,12.966860,77.761580
51,J. P. Nagar,0.020000,0,12.908310,77.590240
94,"Rajarajeshwari Nagar, Bangalore",0.031250,0,12.931780,77.526680


In [84]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bl_merged['Latitude'], bl_merged['Longitude'], bl_merged['Neighborhood'], bl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [85]:
bl_merged.loc[bl_merged['Cluster Labels'] == 0]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
0,Adugodi,0.020000,0,12.944020,77.608000
90,Peenya,0.041667,0,13.031850,77.526790
87,Nayandahalli,0.050000,0,12.942050,77.521000
86,Nandini Layout,0.034483,0,13.014810,77.538910
84,Muthyalanagar,0.025641,0,13.041030,77.551480
47,Hulimavu,0.034483,0,12.880640,77.601470
106,"Siddapura, Bangalore",0.035714,0,12.956130,77.731960
48,Immadihalli,0.062500,0,12.966860,77.761580
51,J. P. Nagar,0.020000,0,12.908310,77.590240
94,"Rajarajeshwari Nagar, Bangalore",0.031250,0,12.931780,77.526680


In [86]:
bl_merged.loc[bl_merged['Cluster Labels'] == 1]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
78,Marathahalli,0.000000,1,12.954660,77.707520
92,R. T. Nagar,0.000000,1,13.024450,77.595900
91,Puttenahalli,0.000000,1,12.894340,77.585070
69,Koramangala,0.000000,1,12.920040,77.625460
89,Palace Guttahalli,0.010417,1,12.995505,77.574771
88,Padmanabhanagar,0.000000,1,12.915470,77.553160
83,Murugeshpalya,0.000000,1,12.955650,77.653350
71,Kumaraswamy Layout,0.000000,1,12.898190,77.559270
72,Kundalahalli,0.000000,1,12.967520,77.715000
82,"Murphy Town, Bangalore",0.000000,1,12.979530,77.624080


In [87]:
bl_merged.loc[bl_merged['Cluster Labels'] == 2]

,Neighborhood,Multiplex,Cluster Labels,Latitude,Longitude
53,Jalahalli,0.125000,2,13.05450,77.52658
117,Varthur,0.074074,2,12.94348,77.74703
28,Dasarahalli,0.111111,2,13.04224,77.51507
127,Yeswanthpur,0.083333,2,13.02954,77.54022


# Observations:

Most of the Multiplex are concentrated in the central area (Close to IT Parks) of bangalore city, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 3 has very low number of Multiplex in the neighborhoods. This represents a great opportunity and high potential areas to open new Multiplex as there is very little to no competition from existing Multiplex.